# 使用基础的iris数据集制作分类模型
这里使用tf2中的高级封装后API：Keras来实现网络

In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from sklearn import datasets
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## 1、载入数据

In [2]:
x_train = datasets.load_iris().data
y_train = datasets.load_iris().target

格式化输出，便于观察（非必要，这里是为了方便讲解）

In [3]:
x_data = pd.DataFrame(x_train, columns=['花萼长度', '花萼宽度', '花瓣长度', '花瓣宽度']) # 为表格增加行索引（左侧）和列标签（上方）
pd.set_option('display.unicode.east_asian_width', True)  # 设置列名对齐
x_data['类别'] = y_train  # 新加一列，列标签为‘类别’，数据为y_data
x_data.to_csv('Iris.csv')
print("x_data add a column: \n", x_data)

x_data add a column: 
      花萼长度  花萼宽度  花瓣长度  花瓣宽度  类别
0         5.1       3.5       1.4       0.2     0
1         4.9       3.0       1.4       0.2     0
2         4.7       3.2       1.3       0.2     0
3         4.6       3.1       1.5       0.2     0
4         5.0       3.6       1.4       0.2     0
..        ...       ...       ...       ...   ...
145       6.7       3.0       5.2       2.3     2
146       6.3       2.5       5.0       1.9     2
147       6.5       3.0       5.2       2.0     2
148       6.2       3.4       5.4       2.3     2
149       5.9       3.0       5.1       1.8     2

[150 rows x 5 columns]


## 2、调整数据格式
这里只做了数据随机化，由于keras支持从numpy直接加载数据，所以不用将数据再转化为tensor张量

In [4]:
np.random.seed(116)
np.random.shuffle(x_train)
np.random.seed(116)
np.random.shuffle(y_train)
tf.random.set_seed(116)

## 3、设置模型结构
使用Sequential函数来定义模型,Sequential函数可以直接以列表形式定义网络结构。适合规模小的网络
这里设计了两个神经元模型，原始数据是四个特征数据对应一个输出特征。
第一个神经元四输入四输出，第二个神经元四输出三输出。

In [5]:
class_flag = True

if class_flag is False:
    model = tf.keras.models.Sequential([
                    tf.keras.layers.Dense(4, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2()),
                    tf.keras.layers.Dense(3, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2())])

3-2 使用class类来定义模型，适合规模大的网络
Dense函数：
    units：维数（神经元个数）
    activation：激活函数，可选：relu softmax sigmoid tanh
    kernel_regularizer：正则化函数

Conv2D:
    filters:卷积核个数
    kernel_size：卷积核尺寸
    strides：卷积核步长
    padding：可填 'valid'  'same'


In [6]:
class IrisModel(Model):    # 继承from tensorflow.keras import Model 作为父类
    def __init__(self):
        super(IrisModel, self).__init__()   # 初始化父类的参数
        
        self.d1 = layers.Dense(units=4, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L2())
        self.d2 = layers.Dense(units=3, activation=tf.keras.activations.softmax, kernel_regularizer=tf.keras.regularizers.L2())
        self.c1 = layers.Conv2D(filters=3, kernel_size=3, strides=1, padding='valid')

    def call(self, input):  # 重写前向传播函数
        y = self.d1(input)
        y = self.d2(y)
        return y

if class_flag:
    model = IrisModel()

## 4+5、设置训练参数并开始训练
```
compile：训练属性设置
    optimizer：参数优化器 
        SGD:        tf.keras.optimizers.SGD(learning_rate=0.1,momentum=动量参数) learning_rate学习率，momentum动量参数
        AdaGrad:    tf.keras.optimizers.Adagrad(learning_rate=学习率)
        Adam:       tf.keras.optimizers.Adam(learning_rate=学习率 , beta_1=0.9, beta_2=0.999)
    loss：损失函数
        MSE:        tf.keras.losses.MeanSquaredError()
        交叉熵损失： tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) from_logits=true时输出值经过一次softmax概率归一化
    metrics：准确率计算方式，就是输出数据类型和标签数据类型如何对应
        数值型（两个都是序列值）：    'accuracy'
        都是独热码：    'categorical_accuracy'
        标签是数值，输出是独热码： 'sparse_categorical_accuracy'

fit: 训练
    输入训练特征数据，标签数据，单次输入数据量，迭代次数
    validation_split=从训练集划分多少比例数据用来测试 /  validation_data=(测试特征数据，测试标签数据)   二选一
    validation_freq=多少次epoch测试一次

summary() 打印网络结构信息
```

In [7]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=500, validation_split=0.2, validation_freq=20)
model.summary()

Epoch 1/500
4/4 [==============================] - 2s 0s/step - loss: 1.7891 - sparse_categorical_accuracy: 0.3000
Epoch 2/500
4/4 [==============================] - 0s 5ms/step - loss: 1.3278 - sparse_categorical_accuracy: 0.3000
Epoch 3/500
4/4 [==============================] - 0s 0s/step - loss: 1.1800 - sparse_categorical_accuracy: 0.3000
Epoch 4/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1315 - sparse_categorical_accuracy: 0.3000
Epoch 5/500
4/4 [==============================] - 0s 0s/step - loss: 1.1088 - sparse_categorical_accuracy: 0.2000
Epoch 6/500
4/4 [==============================] - 0s 0s/step - loss: 1.0920 - sparse_categorical_accuracy: 0.1417
Epoch 7/500
4/4 [==============================] - 0s 5ms/step - loss: 1.0680 - sparse_categorical_accuracy: 0.3833
Epoch 8/500
4/4 [==============================] - 0s 0s/step - loss: 1.0523 - sparse_categorical_accuracy: 0.4000
Epoch 9/500
4/4 [==============================] - 0s 0s/step - loss: 1.0345 

## 6、结果可视化


In [8]:
# # 绘制 loss 曲线
# plt.title('Loss Function Curve')  # 图片标题
# plt.xlabel('Epoch')  # x轴变量名称
# plt.ylabel('Loss')  # y轴变量名称
# plt.plot(loss_results, label="$Loss$")  # 逐点画出trian_loss_results值并连线，连线图标是Loss
# plt.legend()  # 画出曲线图标
# plt.show()  # 画出图像
#
# # 绘制 Accuracy 曲线
# plt.title('Acc Curve')  # 图片标题
# plt.xlabel('Epoch')  # x轴变量名称
# plt.ylabel('Acc')  # y轴变量名称
# plt.plot(acc, label="$Accuracy$")  # 逐点画出test_acc值并连线，连线图标是Accuracy
# plt.legend()
# plt.show()